## YOLOv7 모델 활용한 주행환경 객체탐지

* 직접 수집한 Custom Dataset을 구글 드라이브에 업로드 하고, 압축해제 시켜준다.

In [ ]:
%cd /content/drive/MyDrive/customdataset

!unzip -qq "/content/drive/MyDrive/customdataset/customdataset.zip"

/content/drive/MyDrive/customdataset
replace __MACOSX/._dataset-vehicles? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


* git clone 통해 오픈된 yolov7 모델 가져오기

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

* 모델 학습을 위한 필수 라이브러리 및 버전 설치

In [ ]:
!pip install -r /content/drive/MyDrive/customdataset/yolov7/requirements.txt
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 182 kB 72.7 MB/s 
     |████████████████████████████████| 162 kB 78.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 162 kB 81.4 MB/s 
     |████████████████████████████████| 158 kB 80.5 MB/s 
     |████████████████████████████████| 157 kB 82.6 MB/s 
     |████████████████████████████████| 157 kB 78.2 MB/s 
     |████████████████████████████████| 157 kB 85.0 MB/s 
     |████████████████████████████████| 157 kB 83.2 MB/s 
     |████████████████████████████████| 157 kB 85.0 MB/s 
     |████████████████████████████████| 157 kB 86.3 MB/s 
     |████████████████████████████████| 157 kB 85.7 

* 학습에 사용할 GPU 확인

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Oct 13 15:04:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

* Custom Dataset 사용하여 모델 학습

* epoch = 300회

In [ ]:
!python /content/drive/MyDrive/customdataset/yolov7/train.py --workers 8 --device 0 --batch-size 32 --data /content/drive/MyDrive/customdataset/yolov7/data/custom.yaml --img 640 640 --cfg /content/drive/MyDrive/customdataset/yolov7/cfg/training/yolov7.yaml --weights '' --name yolov7 --hyp data/hyp.scratch.p5.yaml

YOLOR 🚀 v0.1-115-g072f76c torch 1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536.1875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=32, bbox_interval=-1, bucket='', cache_images=False, cfg='/content/drive/MyDrive/customdataset/yolov7/cfg/training/yolov7.yaml', data='/content/drive/MyDrive/customdataset/yolov7/data/custom.yaml', device='0', entity=None, epochs=300, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='./yolov7/data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov7', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov76', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=32, upload_dataset=False, v5_metric=False, weights='', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2022-10-13 15:05:20.

* 학습된 weight 중 가장 최적화된 weight인 best.pt로 테스트 수행

In [ ]:
!python /content/drive/MyDrive/customdataset/yolov7/detect.py --weights /content/drive/MyDrive/customdataset/runs/train/yolov76/weights/best.pt --conf 0.25 --img-size 640 --source /도로주행.jpg

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='/도로주행.jpg', update=False, view_img=False, weights=['/content/drive/MyDrive/customdataset/runs/train/yolov76/weights/best_260.pt'])
YOLOR 🚀 v0.1-115-g072f76c torch 1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536.1875MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36503348 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)

In [ ]:
!python /content/drive/MyDrive/customdataset/yolov7/detect.py --weights /content/drive/MyDrive/customdataset/runs/train/yolov76/weights/best.pt --conf 0.25 --img-size 640 --source /drivephoto.jpg

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='/drivephoto.jpg', update=False, view_img=False, weights=['/content/drive/MyDrive/customdataset/runs/train/yolov76/weights/epoch_299.pt'])
YOLOR 🚀 v0.1-115-g072f76c torch 1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536.1875MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36503348 parameters, 6194944 gradients
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwarg

In [ ]:
!python /content/drive/MyDrive/customdataset/yolov7/detect.py --weights /content/drive/MyDrive/customdataset/runs/train/yolov76/weights/best.pt --conf 0.25 --img-size 640 --source !python /content/drive/MyDrive/customdataset/yolov7/detect.py --weights /content/drive/MyDrive/customdataset/runs/train/yolov76/weights/epoch_299.pt --conf 0.25 --img-size 640 --source !python /content/drive/MyDrive/customdataset/yolov7/detect.py --weights /content/drive/MyDrive/customdataset/runs/train/yolov76/weights/epoch_299.pt --conf 0.25 --img-size 640 --source /588675_448995_0533.jpg

usage: detect.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE]
                 [--img-size IMG_SIZE] [--conf-thres CONF_THRES]
                 [--iou-thres IOU_THRES] [--device DEVICE] [--view-img]
                 [--save-txt] [--save-conf] [--nosave]
                 [--classes CLASSES [CLASSES ...]] [--agnostic-nms]
                 [--augment] [--update] [--project PROJECT] [--name NAME]
                 [--exist-ok] [--no-trace]
detect.py: error: unrecognized arguments: /content/drive/MyDrive/customdataset/yolov7/detect.py /content/drive/MyDrive/customdataset/yolov7/detect.py
